In [1]:
!pip install ftfy regex tqdm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.6 MB/s 


In [2]:
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-df2z2boz
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-df2z2boz
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=5c540382f9e5eb9c3e57ae3c8ab214beb500e850e8f15951f647eaab48b88a28
  Stored in directory: /tmp/pip-ephem-wheel-cache-fzqf0w9t/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [3]:
import torch
import clip 
from PIL import Image 

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cuda


In [4]:
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 223MiB/s]


In [5]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 80.1 MB/s 
     |████████████████████████████████| 6.6 MB 64.5 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 7.6 MB/s 
     |████████████████████████████████| 1.1 MB 81.3 MB/s 
     |████████████████████████████████| 212 kB 94.3 MB/s 
     |████████████████████████████████| 140 kB 74.2 MB/s 
     |████████████████████████████████| 127 kB 93.0 MB/s 
     |████████████████████████████████| 144 kB 80.6 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |█████████████

In [6]:
from datasets import load_dataset

dataset = load_dataset("mnist")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to /root/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [8]:
mnist = dataset['test']

In [9]:
mnist[1]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7FF02EBA5ED0>,
 'label': 2}

In [10]:
mnist[1]['image']

In [11]:
mnist[1]['label']

2

## Experiment $1: Naive-prompt on MNIST dataset (Model: ViT-B32)

In [12]:
image = preprocess(mnist[1]['image']).unsqueeze(0).to(device)

label_list = ["number 0","number 1","number 2","number 3","number 4","number 5","number 6","number 7","number 8","number 9"]

label_list = ["0","1","2","3","4","5","6","7","8","9"]

label_list = []

for i in range(10):
  strr = f"A handwritten number {i}"
  label_list.append(strr)

print(label_list)
text = clip.tokenize(label_list).to(device)

['A handwritten number 0', 'A handwritten number 1', 'A handwritten number 2', 'A handwritten number 3', 'A handwritten number 4', 'A handwritten number 5', 'A handwritten number 6', 'A handwritten number 7', 'A handwritten number 8', 'A handwritten number 9']


In [13]:
with torch.no_grad():
  image_features = model.encode_image(image)
  text_features = model.encode_text(text)

  logits_per_image, logits_per_text = model(image, text)
  probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  print("Label probs:",probs)
  #print("Predict Label:" ,probs.index(max(probs[0]))

Label probs: [[0.1627  0.0445  0.02536 0.06573 0.0722  0.08575 0.0745  0.2683  0.0957
  0.1051 ]]


In [14]:
import numpy as np 

probs

print(np.argmax(probs[0]))

7


## MNIST로 Zero-shot Image Classification 직접 실험 

In [15]:
mnist = dataset['test']

print(mnist)


Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})


In [16]:
model, preprocess = clip.load("ViT-B/32", device=device)


In [17]:
label_list = []

for i in range(10):
  strr = f"A handwritten number {i}"
  label_list.append(strr)

print(label_list)
text = clip.tokenize(label_list).to(device)

['A handwritten number 0', 'A handwritten number 1', 'A handwritten number 2', 'A handwritten number 3', 'A handwritten number 4', 'A handwritten number 5', 'A handwritten number 6', 'A handwritten number 7', 'A handwritten number 8', 'A handwritten number 9']


In [29]:
import numpy as np 

TOTAL = len(mnist)
CORRECT = 0


for i in range(len(mnist)):
  with torch.no_grad():
    image = preprocess(mnist[i]['image']).unsqueeze(0).to(device)

    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    predict = np.argmax(probs[0])

    if predict == mnist[i]['label']: CORRECT = CORRECT + 1

  if (i % 1000 == 0 and i > 0) or i == 10000:
    print(f'Accuracy: {CORRECT}/ {i} ({CORRECT*100.0//i})')

Accuracy: 495/ 1000 (49.0)
Accuracy: 993/ 2000 (49.0)
Accuracy: 1509/ 3000 (50.0)
Accuracy: 2004/ 4000 (50.0)
Accuracy: 2496/ 5000 (49.0)
Accuracy: 3026/ 6000 (50.0)
Accuracy: 3565/ 7000 (50.0)
Accuracy: 4095/ 8000 (51.0)
Accuracy: 4635/ 9000 (51.0)


## Experiment $2: Standard Prompts for zeroshot results on MNIST dataset (Model: ViT-B32 )

In [30]:
image = preprocess(mnist[1]['image']).unsqueeze(0).to(device)

label_list = []

for i in range(10):
  strr = f'a photo of the number: "{i}".'
  label_list.append(strr)

print(label_list)
text = clip.tokenize(label_list).to(device)

['a photo of the number: "0".', 'a photo of the number: "1".', 'a photo of the number: "2".', 'a photo of the number: "3".', 'a photo of the number: "4".', 'a photo of the number: "5".', 'a photo of the number: "6".', 'a photo of the number: "7".', 'a photo of the number: "8".', 'a photo of the number: "9".']


In [31]:

TOTAL = len(mnist)
CORRECT = 0


for i in range(len(mnist)):
  with torch.no_grad():
    image = preprocess(mnist[i]['image']).unsqueeze(0).to(device)

    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    predict = np.argmax(probs[0])

    if predict == mnist[i]['label']: CORRECT = CORRECT + 1

  if (i % 1000 == 0 and i > 0) or i == 10000:
    print(f'Accuracy: {CORRECT}/ {i} ({CORRECT*100.0//i})')

Accuracy: 495/ 1000 (49.0)
Accuracy: 993/ 2000 (49.0)
Accuracy: 1509/ 3000 (50.0)
Accuracy: 2004/ 4000 (50.0)
Accuracy: 2496/ 5000 (49.0)
Accuracy: 3026/ 6000 (50.0)
Accuracy: 3565/ 7000 (50.0)
Accuracy: 4095/ 8000 (51.0)
Accuracy: 4635/ 9000 (51.0)


## Experiment $3: Standard Prompts for zeroshot results on MNIST dataset (Model: ResNet50 )

In [32]:
model, preprocess = clip.load("RN50", device=device)

In [33]:
image = preprocess(mnist[1]['image']).unsqueeze(0).to(device)

label_list = []

for i in range(10):
  strr = f'a photo of the number: "{i}"'
  label_list.append(strr)

print(label_list)
text = clip.tokenize(label_list).to(device)

['a photo of the number: "0".', 'a photo of the number: "1".', 'a photo of the number: "2".', 'a photo of the number: "3".', 'a photo of the number: "4".', 'a photo of the number: "5".', 'a photo of the number: "6".', 'a photo of the number: "7".', 'a photo of the number: "8".', 'a photo of the number: "9".']


In [34]:

TOTAL = len(mnist)
CORRECT = 0


for i in range(len(mnist)):
  with torch.no_grad():
    image = preprocess(mnist[i]['image']).unsqueeze(0).to(device)

    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    predict = np.argmax(probs[0])

    if predict == mnist[i]['label']: CORRECT = CORRECT + 1

  if (i % 1000 == 0 and i > 0) or i == 9999:
    print(f'Accuracy: {CORRECT}/ {i} ({CORRECT*100.0//i})')

Accuracy: 588/ 1000 (58.0)
Accuracy: 1169/ 2000 (58.0)
Accuracy: 1748/ 3000 (58.0)
Accuracy: 2325/ 4000 (58.0)
Accuracy: 2928/ 5000 (58.0)
Accuracy: 3538/ 6000 (58.0)
Accuracy: 4141/ 7000 (59.0)
Accuracy: 4712/ 8000 (58.0)
Accuracy: 5293/ 9000 (58.0)


## Experiment $4: Standard Prompts for zeroshot results on MNIST dataset (Model: ResNet101 )

In [35]:
model, preprocess = clip.load("RN101", device=device)

In [41]:
image = preprocess(mnist[1]['image']).unsqueeze(0).to(device)

label_list = []

for i in range(10):
  strr = f'a photo of the number: "{i}"'
  label_list.append(strr)

print(label_list)
text = clip.tokenize(label_list).to(device)

['a photo of the number: "0"', 'a photo of the number: "1"', 'a photo of the number: "2"', 'a photo of the number: "3"', 'a photo of the number: "4"', 'a photo of the number: "5"', 'a photo of the number: "6"', 'a photo of the number: "7"', 'a photo of the number: "8"', 'a photo of the number: "9"']


In [42]:

TOTAL = len(mnist)
CORRECT = 0


for i in range(len(mnist)):
  with torch.no_grad():
    image = preprocess(mnist[i]['image']).unsqueeze(0).to(device)

    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    predict = np.argmax(probs[0])

    if predict == mnist[i]['label']: CORRECT = CORRECT + 1

  if i % 1000 == 0 and i > 0 or i == 9999:
    print(f'Accuracy: {CORRECT}/ {i} ({CORRECT*100.0//i})')

Accuracy: 513/ 1000 (51.0)
Accuracy: 1041/ 2000 (52.0)
Accuracy: 1580/ 3000 (52.0)
Accuracy: 2098/ 4000 (52.0)
Accuracy: 2622/ 5000 (52.0)
Accuracy: 3186/ 6000 (53.0)
Accuracy: 3757/ 7000 (53.0)
Accuracy: 4322/ 8000 (54.0)
Accuracy: 4894/ 9000 (54.0)
Accuracy: 5417/ 9999 (54.0)
